In [53]:
import pandas as pd
import numpy as np

In [54]:
def oil_database(df):
	df = pd.read_excel("rbf_data_4.xlsx")
	df.iloc[6, 3] = 2.11
	df_op = df.drop(columns=["REFERENCE"])
	column_ref = df["REFERENCE"].astype(float)
	return df_op, column_ref

   X1  X2  X3 REFERENCE
0   1   1   1      2.18
1   1   1   0      1.83
2   0   3   1      1.31
3   1   1   2      2.79
4   2   1   1      2.49
5   2   0   4      3.95
6   1   2   0      2.11
   X1  X2  X3
0   1   1   1
1   1   1   0
2   0   3   1
3   1   1   2
4   2   1   1
5   2   0   4
6   1   2   0
0    2.18
1    1.83
2    1.31
3    2.79
4    2.49
5    3.95
6    2.11
Name: REFERENCE, dtype: float64


In [55]:
def select_two_poles(df_op):
	pole_size = len(df_op.columns)
	pole_zero = np.zeros(pole_size)
	pole_one = df_op.max().values
	nbr_of_poles = 2
	return nbr_of_poles, pole_zero, pole_one

3
[0. 0. 0.]
[2 3 4]


In [56]:
def calculate_euclidian_distance(pole_zero, pole_one):
	pole_distance = np.linalg.norm(pole_one - pole_zero)**2
	return pole_distance

28.999999999999996


In [57]:
def calculate_sigma(nbr_of_poles, pole_distance):
	sigma = np.sqrt(pole_distance) / np.sqrt(2 * nbr_of_poles)
	return sigma


2.692582403567252


In [59]:
def gaussian_function(row, pole_zero, pole_one, sigma):
	result1 = np.e**(-1/(2*(sigma**2)) * np.linalg.norm(row - pole_zero)**2)
	result2 = np.e**(-1/(2*(sigma**2)) * np.linalg.norm(row - pole_one)**2)
	return result1, result2

def apply_radial_basis_function(df_op, pole_zero, pole_one, sigma):
	tuple_series = df_op.apply(gaussian_function, axis=1, args=(pole_zero, pole_one, sigma))
	df_main_matrix = pd.DataFrame(tuple_series.tolist(), columns=['Column 1', 'Column 2'])
	df_main_matrix["Column 3"] = 1
	R = df_main_matrix.values
	return R

0     (0.8131037598190377, 0.3807861904186886)
1    (0.8711587695892689, 0.23497506847332428)
2    (0.5017490561548966, 0.40797404404520005)
3     (0.6611377242318554, 0.5375735693311237)
4    (0.6611377242318554, 0.40797404404520005)
5     (0.2517521153523296, 0.5375735693311237)
6    (0.7083424709523611, 0.28898534244340446)
dtype: object
   Column 1  Column 2  Column 3
0  0.813104  0.380786         1
1  0.871159  0.234975         1
2  0.501749  0.407974         1
3  0.661138  0.537574         1
4  0.661138  0.407974         1
5  0.251752  0.537574         1
6  0.708342  0.288985         1
[[0.81310376 0.38078619 1.        ]
 [0.87115877 0.23497507 1.        ]
 [0.50174906 0.40797404 1.        ]
 [0.66113772 0.53757357 1.        ]
 [0.66113772 0.40797404 1.        ]
 [0.25175212 0.53757357 1.        ]
 [0.70834247 0.28898534 1.        ]]


In [60]:
def calculate_pseudo_inverse(R):
	R_pseudo_inv = np.dot(np.linalg.inv(np.dot(R.T, R)), R.T)
	return R_pseudo_inv

[[ 1.16735865  0.16017947 -0.97217274  1.55694355  0.26214723 -1.61337488
  -0.56108128]
 [ 1.26701616 -1.90402784 -1.14423071  3.78218784  0.44818147 -0.30788102
  -2.1412459 ]
 [-1.10836867  0.80108824  1.2204467  -2.36163099 -0.20348833  1.29570875
   1.3562443 ]]


In [61]:
def transform_column_ref_in_matrix(column_ref):
	A = column_ref.to_numpy().reshape(-1, 1)
	return A

[[2.18]
 [1.83]
 [1.31]
 [2.79]
 [2.49]
 [3.95]
 [2.11]]


In [62]:
def calculate_radial_basis_function_constants(R_pseudo_inv, A):
	a = np.dot(R_pseudo_inv, A)
	return a

[[-0.99566916]
 [ 3.71289911]
 [ 1.53262159]]


In [63]:
# Define the truth_gaussian_function
def truth_gaussian_function(row, pole_zero, pole_one, sigma, a):
	first_term = a[0] * np.e**(-1/(2*(sigma**2)) * np.linalg.norm(row - pole_zero)**2)
	second_term = a[1] * np.e**(-1/(2*(sigma**2)) * np.linalg.norm(row - pole_one)**2)
	third_term = a[2]
	result = first_term + second_term + third_term
	return result

In [64]:
# Test truth_gaussian_function

result = df_op.apply(truth_gaussian_function, axis=1, args=(pole_zero, pole_one, sigma, a))
result


0    [2.1368599607927194]
1    [1.5376743928428769]
2    [2.5478119954813305]
3    [2.8703035759284843]
4     [2.389113613771748]
5    [3.2779162023982873]
6    [1.9003202587845505]
dtype: object